In [1]:
import pandas as pd
import numpy as np
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import keras.backend as K

Using TensorFlow backend.


[]


In [2]:
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1

H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [3]:
cohenvec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p', 'rb'))

In [5]:
from keras.models import load_model
model = load_model('../store/weights/single_setup/FriDec1515:40:462017testing/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b : a, '<lambda>' : lambda a, b : a})
#results_folder = '../store/results/single_message_ThuOct1919:10:492017/'

In [7]:
model_dssm = model.layers[3]

In [8]:
X_cohen = cohenvec.X

In [11]:
bs = 100
pool_cdsr = []
for i in range(int(np.floor(len(X_cohen)/bs))+1) :
    pool_cdsr.append(model_dssm.predict(X_cohen[i*bs:(i+1)*bs,:]))
embedds = np.concatenate(pool_cdsr)

In [12]:
from sklearn.preprocessing import normalize
embedds = normalize(embedds, 'l2')

In [13]:
scores = np.dot(embedds, embedds.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [14]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.874888616243
ACEInhibitors_processed              0.848592
ADHD_processed                       0.834763
Antihistamines_processed             0.919654
AtypicalAntipsychotics_processed     0.968202
BetaBlockers_processed               0.681681
CalciumChannelBlockers_processed     0.755871
Estrogens_processed                  0.964519
NSAIDS_processed                     0.893797
Opiods_processed                     0.859472
OralHypoglycemics_processed          0.902331
ProtonPumpInhibitors_processed       0.936851
SkeletalMuscleRelaxants_processed    0.770937
Statins_processed                    0.869531
Triptans_processed                   0.978507
dtype: float64
